In [3]:
# UNCOMMENT IF YOU DONT HAVE THESE YET
# !pip install PyQt6
# !pip install pymongo
# !pip install face-recognition
!apt-get update
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,218 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:14 http://security.ubuntu.com/ubuntu jam

## Imports

In [4]:
import sys
from PyQt6.QtCore import Qt, QSize,pyqtSignal
from PyQt6.QtGui import QPixmap, QFont,QColor,QIcon
import pymongo
from PyQt6.QtWidgets import QApplication, QSizePolicy
from PyQt6.QtWidgets import QMainWindow, QWidget, QVBoxLayout, QFormLayout, QLabel, QPushButton, QFileDialog, QMessageBox, QTabWidget, QDialog, QScrollArea, QCheckBox, QGridLayout, QDialogButtonBox
import face_recognition
from gridfs import GridFS
from google.colab import drive
import os
import pandas as pd
import shutil
import hashlib
import numpy as np
from pyvirtualdisplay import Display


## initializations

In [5]:
drive.mount('/content/drive')
uploaded_images_folder = '/content/drive/My Drive/uploaded_images'
labelled_images_folder = '/content/drive/My Drive/labelled_images'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
display = Display(visible=0, size=(800, 600))
display.start()

## Styles

In [4]:
class StyleSheet:
    @staticmethod
    def style():
        return """
            * {
                font-family: 'Impact', sans-serif;
                background-color: #1e1e1e;
                color: white; /* Makes all text white */
            }
            QPushButton {
                color: white;
                border-radius: 10px;
                padding: 10px;
                font-size: 16px;
            }
            QPushButton{
                background-color: #BB86FC;
            }
            QPushButton:hover {
                background-color: #D9A4FF;
            }
            QTabBar::tab {
                color: black; /* Sets the tab text color to black */
            }
        """
    def style_labelled_images_page():
        return """
            QMainWindow {
                background-color: #2b2b2b;
                color: white;
            }
            QPushButton {
                background-color: #4b4b4b;
                color: white;
                font-size: 14px;
                padding: 10px;
                border: none;
                margin: 10px;
            }
            QPushButton:hover {
                background-color: #6b6b6b;
            }
            QListWidget {
                background-color: #3b3b3b;
                color: white;
                font-size: 16px;
                margin: 10px;
            }
            QListWidget::item {
                padding: 10px;
            }
            QListWidget::item:selected {
                background-color: #5b5b5b;
            }
            QLabel {
                font-size: 16px;
                color: white;
            }
        """

## Widgets

In [5]:

class ImageLabel(QWidget):
    image_loaded = pyqtSignal(bool)  # Signal to emit after loading names

    def __init__(self, image_path, face_locations, face_encodings):
        super().__init__()
        self.image_path = image_path
        self.dest_path_csv_file = image_path
        self.labelPic = QLabel(self)
        self.original_image = QPixmap(image_path)
        self.face_locations = face_locations
        self.face_encodings = face_encodings
        self.face_names = ["Unknown"] * len(face_locations)  # Initialize with "Unknown"
        self.move_image = False

        self.initUI()
        self.load_names()

    def initUI(self):
        self.setSizePolicy(QSizePolicy.Policy.Expanding, QSizePolicy.Policy.Expanding)
        layout = QVBoxLayout()
        layout.addWidget(self.labelPic)
        self.setLayout(layout)
        self.update_image()

    def update_image(self):
        # Scale the image to fit the label size
        scaled_image = self.original_image.scaled(self.labelPic.size(), Qt.AspectRatioMode.KeepAspectRatio, Qt.TransformationMode.SmoothTransformation)
        self.labelPic.setPixmap(scaled_image)
        self.labelPic.adjustSize()

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.update_image()

    def paintEvent(self, event):
        # Create a COPY for drawing
        image_copy = self.original_image.copy()

        scaled_image = image_copy.scaled(self.labelPic.size(), Qt.AspectRatioMode.KeepAspectRatio, Qt.TransformationMode.SmoothTransformation)

        # Draw the scaled image
        image_for_drawing = scaled_image.toImage()
        painter = QPainter(image_for_drawing)

        # Draw rectangles and names
        painter.setPen(QPen(Qt.GlobalColor.green, 2))
        painter.setFont(QFont('Arial', 14))

        # Get the scale factors
        scaled_image_size = scaled_image.size()
        width_scale = scaled_image_size.width() / self.original_image.width()
        height_scale = scaled_image_size.height() / self.original_image.height()

        self.scaled_face_locations = []
        for index, (top, right, bottom, left) in enumerate(self.face_locations):
            # Scale face locations to fit the scaled image
            scaled_top = int(top * height_scale)
            scaled_right = int(right * width_scale)
            scaled_bottom = int(bottom * height_scale)
            scaled_left = int(left * width_scale)

            self.scaled_face_locations.append(QRect(scaled_left, scaled_top, scaled_right - scaled_left, scaled_bottom - scaled_top))

            # White Background Behind Text
            text_rect = painter.boundingRect(self.scaled_face_locations[-1], Qt.AlignmentFlag.AlignLeft, self.face_names[self.scaled_face_locations.index(self.scaled_face_locations[-1])])
            text_rect.setHeight(text_rect.height() + 7)
            text_rect.moveTop(text_rect.top() - text_rect.height() - 5) # 5 is a padding value

            painter.drawRect(self.scaled_face_locations[-1])

            painter.fillRect(text_rect, QBrush(Qt.GlobalColor.white))

            painter.drawText(self.scaled_face_locations[-1].left(), self.scaled_face_locations[-1].top() - 10, self.face_names[index])  # Correctly use index

        painter.end()

        # Set the scaled image on the label
        self.labelPic.setPixmap(QPixmap.fromImage(image_for_drawing))
        self.labelPic.resize(self.labelPic.pixmap().size())

    def mousePressEvent(self, event):
        click_x = int(event.position().x())
        click_y = int(event.position().y())

        for i, rect in enumerate(self.scaled_face_locations):
            if rect.contains(click_x, click_y):
                self.edit_name(i)
                return

    def get_all_face_encodings(self, csv_file):
        try:
            df = pd.read_csv(csv_file)
            if df.empty:
                return []
            encodings = [np.array(eval(encoding)) for encoding in df['face_encoding']]
            return encodings
        except (pd.errors.EmptyDataError, FileNotFoundError):
            return []

    def load_names(self):
        # If the image does not contain any faces then return
        if len(self.face_locations) < 1:
            return

        csv_file = 'faces_data.csv'  # Path to your CSV file
        if not os.path.exists(csv_file):
            return

        df = pd.read_csv(csv_file)
        if df.empty:
            return

        all_faces_encodings = self.get_all_face_encodings(csv_file)
        for i, face_encoding in enumerate(self.face_encodings):
            if all_faces_encodings:
                distances = face_recognition.face_distance(all_faces_encodings, face_encoding)
                best_match_index = np.argmin(distances)
                if distances[best_match_index] <= 0.55:
                    self.face_names[i] = df.iloc[best_match_index]['name']
        print("Loaded names:", self.face_names)  # Debug print

        self.rename_images()
        self.save_image_names()  # Save image and face names to CSV

        # Move the image if there are no "Unknown" names
        if not any(name == "Unknown" for name in self.face_names):
            self.move_image = True
        self.image_loaded.emit(self.move_image)  # Emit signal with the value


    def rename_images(self):
        item_text = os.path.basename(self.image_path)
        src_path = os.path.join(uploaded_images_folder, item_text)

        # Check if the item exists in the uploaded_images folder
        if not os.path.exists(src_path):
            return

        dest_path = os.path.join(labelled_images_folder, item_text)

        # Create the folder if it doesn't exist
        labelled_folder = labelled_images_folder
        if not os.path.exists(labelled_folder):
            os.makedirs(labelled_folder)

        # Check if the same content image already exists
        same_content = False
        for dest_filename in os.listdir(labelled_images_folder):
            full_dest_path = os.path.join(labelled_images_folder, dest_filename)
            if open(src_path, 'rb').read() == open(full_dest_path, 'rb').read():
                same_content = True
                break

        if os.path.exists(dest_path) and not same_content:
            base_name, extension = os.path.splitext(item_text)
            counter = 1
            new_dest_path = os.path.join(labelled_images_folder, f"{base_name} ({counter}){extension}")
            while os.path.exists(new_dest_path):
                counter += 1
                new_dest_path = os.path.join(labelled_images_folder, f"{base_name} ({counter}){extension}")
            dest_path = new_dest_path
            self.dest_path_csv_file = dest_path

    def edit_name(self, index):
        csv_file = 'faces_data.csv'  # Path to your CSV file
        try:
            df = pd.read_csv(csv_file)
        except (pd.errors.EmptyDataError, FileNotFoundError):
            df = pd.DataFrame(columns=['name', 'face_encoding'])

        images_csv_file = 'image_face_names.csv'
        try:
            im_df = pd.read_csv(images_csv_file)
        except (pd.errors.EmptyDataError, FileNotFoundError):
            im_df = pd.DataFrame(columns=['image_name', 'face_names'])

        # Find if the face is already in our CSV
        face_encoding = self.face_encodings[index]
        all_faces_encodings = self.get_all_face_encodings(csv_file)
        matched_name = None
        if all_faces_encodings:
            distances = face_recognition.face_distance(all_faces_encodings, face_encoding)
            best_match_index = np.argmin(distances)
            if distances[best_match_index] <= 0.55:
                matched_name = df.iloc[best_match_index]['name']
                reply = QMessageBox.question(self, 'Match Found', f"The name for this person is {matched_name}. Do you want to edit it?", QMessageBox.StandardButton.Yes | QMessageBox.StandardButton.No, QMessageBox.StandardButton.No)
                if reply == QMessageBox.StandardButton.No:
                    return
                # Delete the row with the matched name if answer is yes
                df = df.drop(best_match_index).reset_index(drop=True)

        name, ok = QInputDialog.getText(self, 'Edit Name', 'Enter the name:')
        if ok and name:
            if matched_name:
                # Replace the old name with the new name in all entries
                im_df['face_names'] = im_df['face_names'].apply(lambda x: ', '.join([name if each_name.strip() == matched_name else each_name.strip() for each_name in x.split(',')]))
                im_df.to_csv(images_csv_file, index=False)
            new_entry = pd.DataFrame({
                "name": [name],
                "face_encoding": [face_encoding.tolist()]
            })
            df = pd.concat([df, new_entry], ignore_index=True)
            df.to_csv(csv_file, index=False)
            self.face_names[index] = name  # Update the name for the face
            print(f"Updated name at index {index}: {name}")  # Debug print
            self.load_names()  # Reload names from CSV after update
            self.update()  # Repaint the widget to show the updated name
            self.save_image_names()  # Save image and face names to CSV

    def save_image_names(self):
        image_name = os.path.basename(self.dest_path_csv_file)  # Get the image file name
        data = {
            "image_name": image_name,
            "face_names": ", ".join(self.face_names)  # Join all face names into a single string
        }

        csv_file = "image_face_names.csv"

        # Check if the file exists
        if os.path.exists(csv_file):
            # Read the existing data
            df = pd.read_csv(csv_file)

            # Check if the image name already exists
            if image_name in df['image_name'].values:
                # Update the existing row
                df.loc[df['image_name'] == image_name, 'face_names'] = data['face_names']
            else:
                # Append a new row
                df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
        else:
            # Create a new DataFrame if the file does not exist
            df = pd.DataFrame([data])

        # Write the DataFrame to the CSV file
        df.to_csv(csv_file, index=False)

        print(f"Saved image names for {image_name}")  # Debug print


In [6]:

class ScrollListExample(QWidget):
    def __init__(self, main_window):
        super().__init__()

        self.main_window = main_window  # Reference to the main window

        self.setWindowTitle('Scrollable List Example')
        self.setGeometry(100, 100, 400, 600)

        layout = QVBoxLayout()

        # Heading
        heading = QLabel("Labeled Photos")
        heading.setAlignment(Qt.AlignmentFlag.AlignCenter)
        heading.setFont(QFont('Arial', 20))
        heading.setStyleSheet("QLabel { color: blue; margin: 20px; }")
        layout.addWidget(heading)

        self.listWidget = QListWidget()
        self.listWidget.setStyleSheet("QListWidget { margin: 10px; }")

        # Connect to MongoDB
        client = pymongo.MongoClient('mongodb://localhost:27017/')
        db = client['photo_database']
        fs = GridFS(db)

        # Fetch data from MongoDB
        people = db.people.find()

        for person in people:
            item = QListWidgetItem()
            widget = QWidget()
            h_layout = QHBoxLayout()

            label_name = QLabel(person['name'])
            label_name.setFont(QFont('Arial', 14))
            label_image = QLabel()

            # Load image from MongoDB GridFS
            image_id = person['image_id']
            image_data = fs.get(image_id).read()
            pixmap = QPixmap()
            if pixmap.loadFromData(image_data):
                pixmap = pixmap.scaled(100, 100, Qt.AspectRatioMode.KeepAspectRatio, Qt.TransformationMode.SmoothTransformation)
            else:
                # Placeholder image if the file is not found
                pixmap = QPixmap(100, 100)
                pixmap.fill(QColor('gray'))
                label_image.setText("Image not found")

            label_image.setPixmap(pixmap)

            search_button = QPushButton("Search")
            search_button.clicked.connect(self.on_search_button_clicked)
            search_button.setStyleSheet("QPushButton { margin-left: 10px; }")

            h_layout.addWidget(label_image)
            h_layout.addWidget(label_name)
            h_layout.addWidget(search_button)
            widget.setLayout(h_layout)

            item.setSizeHint(widget.sizeHint())
            self.listWidget.addItem(item)
            self.listWidget.setItemWidget(item, widget)

        layout.addWidget(self.listWidget)

        back_button = QPushButton("Back")
        back_button.clicked.connect(self.go_back)  # Go back to the main window
        back_button.setStyleSheet("QPushButton { margin: 10px; }")
        layout.addWidget(back_button)

        self.setLayout(layout)

    def on_search_button_clicked(self):
        # Placeholder for search button functionality
        print("Search button clicked")

    def go_back(self):
        self.hide()
        self.main_window.show()


In [7]:

class SlideshowWidget(QWidget):
    def __init__(self):
        super().__init__()

        self.image_label = QLabel("No Image")
        self.image_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.image_label.setFixedSize(600, 400)  # Fixed size for displaying images
        self.image_label.setScaledContents(True)
        self.image_paths = []
        self.current_index = 0

        self.setLayout(QVBoxLayout())
        self.layout().addWidget(self.image_label)

        self.prev_button = QPushButton("Before")
        self.prev_button.clicked.connect(self.prev_image)
        self.prev_button.setEnabled(False)  # Initially disable prev button

        self.next_button = QPushButton("After")
        self.next_button.clicked.connect(self.next_image)
        self.next_button.setEnabled(False)  # Initially disable next button

        button_layout = QHBoxLayout()
        button_layout.addWidget(self.prev_button)
        button_layout.addStretch(1)
        button_layout.addWidget(self.next_button)

        self.layout().addLayout(button_layout)

    def set_image_paths(self, image_paths):
        self.image_paths = image_paths
        if self.image_paths:
            self.show_image()
            self.update_buttons()

    def show_image(self):
        pixmap = QPixmap(self.image_paths[self.current_index])
        pixmap = pixmap.scaledToWidth(self.image_label.width())  # Scale pixmap to fit label width
        self.image_label.setPixmap(pixmap)

    def next_image(self):
        if self.current_index < len(self.image_paths) - 1:
            self.current_index += 1
            self.show_image()
            self.update_buttons()

    def prev_image(self):
        if self.current_index > 0:
            self.current_index -= 1
            self.show_image()
            self.update_buttons()

    def update_buttons(self):
        self.prev_button.setEnabled(self.current_index > 0)
        self.next_button.setEnabled(self.current_index < len(self.image_paths) - 1)

## Pages

In [8]:


class LabellingPic(QMainWindow):
    def __init__(self, image_file_name, folder):
        super().__init__()
        self.image_file_name = image_file_name
        self.setGeometry(200, 200, 800, 600)  # Adjust window size as needed
        self.setWindowTitle("Labelling Pictures")
        self.folder_name = folder
        self.final_move_image = False
        self.initUI()

    def initUI(self):
        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)

        # Create a vertical layout for the main window
        main_layout = QVBoxLayout(central_widget)
        main_layout.setContentsMargins(10, 40, 10, 10)

        # Label
        self.label = QLabel()
        self.label.setText("Label the Pictures below: ")
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.label.setFont(QFont('Arial', 20))
        main_layout.addWidget(self.label)

        # Load image and detect faces
        image_dir = self.folder_name
        image_path = os.path.join(image_dir, self.image_file_name)
        image = face_recognition.load_image_file(image_path)
        face_locations = face_recognition.face_locations(image)
        face_encodings = face_recognition.face_encodings(image)

        # Create custom widget for displaying image with rectangles
        self.image_label = ImageLabel(image_path, face_locations, face_encodings)
        self.final_move_image = self.image_label.move_image
        self.image_label.image_loaded.connect(self.update_final_move_image)  # Connect the signal

        # Add the image_label widget to the main layout
        main_layout.addWidget(self.image_label)

        # Set size policy to expand with window size
        self.image_label.setSizePolicy(QSizePolicy.Policy.Expanding, QSizePolicy.Policy.Expanding)

    def update_final_move_image(self, move_image):
        self.final_move_image = move_image


In [9]:

class AllImagesWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle('All Images')
        self.setGeometry(100, 100, 800, 600)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout()
        self.central_widget.setLayout(self.layout)

        self.listWidget = QListWidget()
        self.listWidget.setIconSize(QSize(100, 100))
        self.listWidget.setStyleSheet("QListWidget { margin: 10px; }")

        self.load_images_from_folder(uploaded_images_folder)

        self.layout.addWidget(self.listWidget)

        back_button = QPushButton("Back")
        back_button.clicked.connect(self.go_back)
        back_button.setStyleSheet("QPushButton { margin: 10px; }")
        self.layout.addWidget(back_button)

        show_button = QPushButton("Show")
        show_button.clicked.connect(self.show_item)
        self.layout.addWidget(show_button)

        delete_button = QPushButton("Delete")
        delete_button.clicked.connect(self.delete_item)
        self.layout.addWidget(delete_button)

        # Connect double-click signal to the slot
        self.listWidget.itemDoubleClicked.connect(self.item_double_clicked)

    def refresh_list(self):
        self.listWidget.clear()
        self.load_images_from_folder(uploaded_images_folder)

    def load_images_from_folder(self, folder_path):
        # Load all images from a folder and display them in the list widget
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                item = QListWidgetItem()
                pixmap = QPixmap(os.path.join(folder_path, filename))
                pixmap = pixmap.scaled(100, 100, Qt.AspectRatioMode.KeepAspectRatio, Qt.TransformationMode.SmoothTransformation)
                item.setIcon(QIcon(pixmap))
                item.setText(filename)
                self.listWidget.addItem(item)

    def show_item(self):
        current_item = self.listWidget.currentItem()
        if current_item:
            item_text = current_item.text()
            print(f"Selected item text: {item_text}")
            # Optionally, show a message box with the selected item's text
            QMessageBox.information(self, "Selected Item", f"Selected item: {item_text}")

    def delete_item(self):
        current_item = self.listWidget.currentItem()
        if current_item:
            item_text = current_item.text()
            reply = QMessageBox.question(self, 'Delete Confirmation',
                                         f"Are you sure you want to delete {item_text}?",
                                         QMessageBox.StandardButton.Yes | QMessageBox.StandardButton.No,
                                         QMessageBox.StandardButton.No)
            if reply == QMessageBox.StandardButton.Yes:
                # Delete the file from the folder
                folder_path = uploaded_images_folder
                file_path = os.path.join(folder_path, item_text)
                if os.path.exists(file_path):
                    os.remove(file_path)
                    self.listWidget.takeItem(self.listWidget.row(current_item))

                    # Remove the image entry from the CSV file
                    self.remove_from_images_CSV(item_text)

                    QMessageBox.information(self, "Delete", f"{item_text} has been deleted.")
                else:
                    QMessageBox.warning(self, "Delete", f"{item_text} not found.")


    def item_double_clicked(self, item):
        current_item = self.listWidget.currentItem()
        if current_item:
            item_text = current_item.text()
            print(f"Double-clicked on: {item_text}")
            self.label_image = LabellingPic(item_text, uploaded_images_folder)
            self.label_image.show()
            self.label_image.closeEvent = lambda event: self.on_labelling_pic_close(event, item_text, self.label_image.final_move_image)

    def on_labelling_pic_close(self, event, item_text, final_move_image):
        if (final_move_image == True):
            self.move_image_to_labelled_folder(item_text)
        self.refresh_list()
        event.accept()

    def move_image_to_labelled_folder(self, item_text):
        src_path = os.path.join(uploaded_images_folder, item_text)
        dest_path = os.path.join(labelled_images_folder, item_text)

        # Create the folder if it doesn't exist
        labelled_folder = labelled_images_folder
        if not os.path.exists(labelled_folder):
            os.makedirs(labelled_folder)

        # Check if the same content image already exists
        same_content = False
        existing_file_path = None
        try:
            for dest_filename in os.listdir(labelled_images_folder):
                full_dest_path = os.path.join(labelled_images_folder, dest_filename)
                if os.path.isfile(full_dest_path):  # Ensure it's a file, not a directory
                    try:
                        with open(src_path, 'rb') as src_file, open(full_dest_path, 'rb') as dest_file:
                            if src_file.read() == dest_file.read():
                                same_content = True
                                existing_file_path = full_dest_path
                                break
                    except PermissionError as e:
                        print(f"Permission error: {e}")
                        QMessageBox.warning(self, "Permission Error", f"Permission denied for file: {full_dest_path}")
                        return
        except Exception as e:
            print(f"Error comparing files: {e}")
            QMessageBox.warning(self, "Error", f"Error comparing files: {e}")
            return

        if same_content:
            replace_reply = QMessageBox.question(self, 'Replace Confirmation',
                                                f"This image already exists in 'labelled images'. Do you want to replace it?",
                                                QMessageBox.StandardButton.Yes | QMessageBox.StandardButton.No,
                                                QMessageBox.StandardButton.No)
            if replace_reply == QMessageBox.StandardButton.Yes:
                if existing_file_path:
                    try:
                        os.remove(existing_file_path)  # Remove the existing image
                        if not (os.path.basename(existing_file_path) == item_text):
                            self.remove_from_images_CSV(os.path.basename(existing_file_path))
                    except PermissionError as e:
                        print(f"Permission error: {e}")
                        QMessageBox.warning(self, "Permission Error", f"Permission denied for file: {existing_file_path}")
                        return
            else:
                if not (os.path.basename(existing_file_path) == item_text):
                    self.remove_from_images_CSV(item_text)
                return

        if os.path.exists(dest_path) and not same_content:
            base_name, extension = os.path.splitext(item_text)
            counter = 1
            new_dest_path = os.path.join(labelled_images_folder, f"{base_name} ({counter}){extension}")
            while os.path.exists(new_dest_path):
                counter += 1
                new_dest_path = os.path.join(labelled_images_folder, f"{base_name} ({counter}){extension}")
            dest_path = new_dest_path

        try:
            shutil.move(src_path, dest_path)
        except PermissionError as e:
            print(f"Permission error: {e}")
            QMessageBox.warning(self, "Permission Error", f"Permission denied for file: {src_path}")
            return

        self.listWidget.takeItem(self.listWidget.row(self.listWidget.findItems(item_text, Qt.MatchFlag.MatchExactly)[0]))
        QMessageBox.information(self, "Move", f"{item_text} has been moved to 'labelled images'.")

    def remove_from_images_CSV(self, item_text):
        # Remove the entry from the CSV file
        csv_file = "image_face_names.csv"
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
            if item_text in df['image_name'].values:
                df = df[df['image_name'] != item_text]
                df.to_csv(csv_file, index=False)

    def go_back(self):
        self.close()


In [10]:
from PyQt6.QtWidgets import QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QListWidget, QListWidgetItem, QMessageBox, QLabel, QLineEdit
from PyQt6.QtCore import Qt, QSize
from PyQt6.QtGui import QPixmap, QIcon
import os
import pandas as pd
import shutil

class LabelledImagesWindow(QMainWindow):
    def __init__(self, main_window):
        super().__init__()

        self.setWindowTitle('Labelled Images')
        self.setGeometry(100, 100, 800, 600)
        self.main_window = main_window

        self.setStyleSheet(StyleSheet.style_labelled_images_page())

        self.all_faces_dict = dict()

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout()
        self.central_widget.setLayout(self.layout)

        self.search_bar = QLineEdit()
        self.search_bar.setPlaceholderText("Search folders...")
        self.search_bar.textChanged.connect(self.filter_folders)
        self.layout.addWidget(self.search_bar)

        self.folder_list_widget = QListWidget()
        self.folder_list_widget.setIconSize(QSize(100, 100))
        self.layout.addWidget(self.folder_list_widget)
        self.folder_list_widget.itemClicked.connect(self.load_images_from_dict)

        self.image_list_widget = QListWidget()
        self.image_list_widget.setIconSize(QSize(100, 100))
        self.layout.addWidget(self.image_list_widget)

        self.image_display_label = QLabel()
        self.layout.addWidget(self.image_display_label)

        self.load_folders(labelled_images_folder)

        button_layout = QHBoxLayout()
        self.layout.addLayout(button_layout)

        back_button = QPushButton("Back")
        back_button.clicked.connect(self.go_back)
        button_layout.addWidget(back_button)

        show_button = QPushButton("Show")
        show_button.clicked.connect(self.show_item)
        button_layout.addWidget(show_button)

        delete_button = QPushButton("Delete")
        delete_button.clicked.connect(self.delete_item)
        button_layout.addWidget(delete_button)

        organize_button = QPushButton("Organize")
        organize_button.clicked.connect(lambda: self.organize_images(self.all_faces_dict, 1))
        self.organize_images(self.all_faces_dict, 0)
        print(self.all_faces_dict)
        button_layout.addWidget(organize_button)

        self.image_list_widget.itemDoubleClicked.connect(self.item_double_clicked)

    def load_folders(self, root_folder):
        self.folder_list_widget.clear()
        for folder_name in list(self.all_faces_dict.keys()):
            item = QListWidgetItem(folder_name)
            item.setData(Qt.ItemDataRole.UserRole, folder_name)
            self.folder_list_widget.addItem(item)

    def load_images_from_dict(self, item):
        folder_path = "labelled_images"
        self.image_list_widget.clear()
        name = self.folder_list_widget.currentItem().text()
        looping_list = self.all_faces_dict[name]

        for filename in looping_list:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                item = QListWidgetItem()
                image_path = os.path.join(folder_path, filename)
                if os.path.exists(image_path):
                    pixmap = QPixmap(image_path)
                    if not pixmap.isNull():
                        pixmap = pixmap.scaled(100, 100, Qt.AspectRatioMode.KeepAspectRatio, Qt.TransformationMode.SmoothTransformation)
                        item.setIcon(QIcon(pixmap))
                        item.setData(Qt.ItemDataRole.UserRole, image_path)
                        self.image_list_widget.addItem(item)
                    else:
                        print(f"Error loading pixmap for: {image_path}")
                else:
                    print(f"Image not found: {image_path}")

    def filter_folders(self):
        search_query = self.search_bar.text().lower()
        for index in range(self.folder_list_widget.count()):
            item = self.folder_list_widget.item(index)
            folder_name = item.text().lower()
            item.setHidden(search_query not in folder_name)

    def show_item(self):
        current_item = self.image_list_widget.currentItem()
        if current_item:
            filename = current_item.data(Qt.ItemDataRole.UserRole)
            pixmap = QPixmap(filename)
            self.image_display_label.setPixmap(pixmap.scaled(self.image_display_label.size(), Qt.AspectRatioMode.KeepAspectRatio))
            QMessageBox.information(self, "Selected Item", f"Selected item: {filename}")

    def delete_item(self):
        current_item = self.image_list_widget.currentItem()
        if current_item:
            filename = current_item.data(Qt.ItemDataRole.UserRole)
            reply = QMessageBox.question(self, 'Delete Confirmation',
                                         f"Are you sure you want to delete {filename}?",
                                         QMessageBox.StandardButton.Yes | QMessageBox.StandardButton.No,
                                         QMessageBox.StandardButton.No)
            if reply == QMessageBox.StandardButton.Yes:
                if os.path.exists(filename):
                    os.remove(filename)
                    self.image_list_widget.takeItem(self.image_list_widget.row(current_item))
                    self.remove_from_images_CSV(os.path.basename(filename))
                    QMessageBox.information(self, "Delete", f"{filename} has been deleted.")
                else:
                    QMessageBox.warning(self, "Delete", f"{filename} not found.")

    def remove_from_images_CSV(self, item_text):
        # Remove the entry from the CSV file
        csv_file = "image_face_names.csv"
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
            if item_text in df['image_name'].values:
                df = df[df['image_name'] != item_text]
                df.to_csv(csv_file, index=False)

    def item_double_clicked(self, item):
        current_item = self.image_list_widget.currentItem()
        if current_item:
            filename = current_item.data(Qt.ItemDataRole.UserRole)
            self.label_image = LabellingPic(filename, "")
            self.label_image.show()
            self.label_image.closeEvent = lambda event: self.refresh_list()

    def refresh_list(self):
        current_item = self.folder_list_widget.currentItem()
        if current_item:
            self.load_images_from_dict(current_item)

    def go_back(self):
        self.main_window.show()
        self.hide()

    def organize_images(self, all_faces_dict, key):
        folder_path = labelled_images_folder
        csv_file = 'image_face_names.csv'
        self.all_faces_dict.clear()
        self.image_list_widget.clear()
        if not os.path.exists(csv_file):
            QMessageBox.warning(self, "Error", "CSV file not found.")
            return

        df = pd.read_csv(csv_file)
        for _, row in df.iterrows():
            image_name = row['image_name']
            face_names = row['face_names'].split(", ")
            for face in face_names:
                if face in all_faces_dict:
                    all_faces_dict[face].append(image_name)
                else:
                    all_faces_dict[face] = [image_name]

        self.load_folders(folder_path)
        if key:
            QMessageBox.information(self, "Organize", "Images have been organized by person.")


In [11]:
class UploadPage(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Image Upload Example")
        self.setGeometry(100, 100, 800, 600)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout()
        self.central_widget.setLayout(self.layout)

        self.tab_widget = QTabWidget()
        self.tab_1 = QWidget()
        self.tab_2 = QWidget()

        self.upload_dir = os.path.join(os.getcwd(), uploaded_images_folder)
        os.makedirs(self.upload_dir, exist_ok=True)

        self.option_upload_single()
        self.option_upload_multiple()

        self.tab_widget.addTab(self.tab_1, "Tab 1")
        self.tab_widget.addTab(self.tab_2, "Tab 2")
        self.layout.addWidget(self.tab_widget)

        self.back = QPushButton("Back")
        self.back.clicked.connect(self.back_prevpage)
        self.layout.addWidget(self.back)

        self.return_back = QPushButton("Return Home")
        self.return_back.clicked.connect(self.return_home)
        self.layout.addWidget(self.return_back)

    def option_upload_single(self):
        layout = QFormLayout()
        self.upload_button_single = QPushButton("Upload Image")
        self.upload_button_single.clicked.connect(self.upload_image)
        layout.addWidget(self.upload_button_single)

        self.image_label_single = QLabel("No Image Uploaded")
        self.image_label_single.setAlignment(Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(self.image_label_single)

        self.tab_1.setLayout(layout)

    def option_upload_multiple(self):
        layout = QFormLayout()
        self.upload_button_multiple = QPushButton("Upload Multiple Images")
        self.upload_button_multiple.clicked.connect(self.upload_images)
        layout.addWidget(self.upload_button_multiple)

        self.tab_2.setLayout(layout)

    def upload_image(self):
        try:
            file_name, _ = QFileDialog.getOpenFileName(self, "Select an Image", "",
                                                       "Image Files (*.png *.jpg *.jpeg *.bmp);;All Files (*)")
            if file_name:
                if self.image_exists(file_name):
                    QMessageBox.warning(self, "Duplicate Image", "This image already exists in the directory.")
                else:
                    selected_files = self.confirm_upload([file_name])
                    if selected_files:
                        pixmap = QPixmap(selected_files[0])
                        self.image_label_single.setPixmap(pixmap.scaled(self.image_label_single.size(), Qt.AspectRatioMode.KeepAspectRatio))
                        self.image_label_single.setText("")
                        self.save_image(selected_files[0])
        except Exception as e:
            QMessageBox.critical(self, "Error", f"An error occurred: {e}")

    def upload_images(self):
        try:
            file_names, _ = QFileDialog.getOpenFileNames(self, "Select Images", "",
                                                         "Image Files (*.png *.jpg *.jpeg *.bmp);;All Files (*)")
            if file_names:
                new_files = [file_name for file_name in file_names if not self.image_exists(file_name)]
                selected_files = self.confirm_upload(new_files)
                if selected_files:
                    for file_name in selected_files:
                        self.save_image(file_name)
                    QMessageBox.information(self, "Images Uploaded", f"Uploaded {len(selected_files)} images.")
                    if len(selected_files) < len(new_files):
                        QMessageBox.warning(self, "Duplicate Images", f"{len(new_files) - len(selected_files)} images were duplicates and were not uploaded.")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"An error occurred: {e}")

    def save_image(self, file_name):
        try:
            base_name = os.path.basename(file_name)
            name, ext = os.path.splitext(base_name)
            destination = os.path.join(self.upload_dir, base_name)
            count = 1

            while os.path.exists(destination):
                new_name = f"{name} ({count}){ext}"
                destination = os.path.join(self.upload_dir, new_name)
                count += 1

            with open(file_name, 'rb') as src_file:
                image_data = src_file.read()
                with open(destination, 'wb') as dest_file:
                    dest_file.write(image_data)
            print(f"Image saved to directory with path: {destination}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Failed to save image: {e}")

    def image_exists(self, file_name):
        try:
            with open(file_name, 'rb') as image_file:
                image_data = image_file.read()
                image_hash = hashlib.sha256(image_data).hexdigest()
                for existing_file in os.listdir(self.upload_dir):
                    existing_file_path = os.path.join(self.upload_dir, existing_file)
                    with open(existing_file_path, 'rb') as existing_image_file:
                        existing_image_data = existing_image_file.read()
                        existing_image_hash = hashlib.sha256(existing_image_data).hexdigest()
                        if existing_image_hash == image_hash:
                            return True
                return False
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Failed to check if image exists: {e}")
            return False

    def return_home(self):
        self.hide()
        self.start = StartingPage()
        self.start.show()

    def back_prevpage(self):
        self.hide()
        self.start = UnlabelledOptionsPage()
        self.start.show()

    def confirm_upload(self, file_names):
        dialog = QDialog(self)
        dialog.setWindowTitle("Confirm Upload")
        dialog.setGeometry(100, 100, 600, 400)

        scroll_area = QScrollArea()
        scroll_area.setWidgetResizable(True)
        scroll_content = QWidget()
        scroll_layout = QGridLayout(scroll_content)
        scroll_content.setLayout(scroll_layout)

        checkboxes = []
        for i, file_name in enumerate(file_names):
            pixmap = QPixmap(file_name)
            image_label = QLabel()
            image_label.setPixmap(pixmap.scaled(100, 100, Qt.AspectRatioMode.KeepAspectRatio))

            checkbox = QCheckBox()
            checkbox.setChecked(True)

            checkboxes.append((checkbox, file_name))
            scroll_layout.addWidget(image_label, i // 4, (i % 4) * 2)
            scroll_layout.addWidget(checkbox, i // 4, (i % 4) * 2 + 1)

        scroll_area.setWidget(scroll_content)

        dialog_layout = QVBoxLayout(dialog)
        dialog_layout.addWidget(scroll_area)

        button_box = QDialogButtonBox(QDialogButtonBox.StandardButton.Ok | QDialogButtonBox.StandardButton.Cancel)
        button_box.accepted.connect(dialog.accept)
        button_box.rejected.connect(dialog.reject)
        dialog_layout.addWidget(button_box)

        result = dialog.exec()

        if result == QDialog.DialogCode.Accepted:
            selected_files = [file_name for checkbox, file_name in checkboxes if checkbox.isChecked()]
            return selected_files
        else:
            return []


In [12]:
class UnlabelledOptionsPage(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Unlabelled Options")
        self.setGeometry(100, 100, 400, 200)
        self.initUI()

    def initUI(self):
        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)
        layout = QVBoxLayout(central_widget)

        self.upload_button = QPushButton("Upload Images")
        self.upload_button.clicked.connect(self.open_upload_page)
        layout.addWidget(self.upload_button)

        self.label_button = QPushButton("Label Images")
        self.label_button.clicked.connect(self.label_images)  # Placeholder functionality
        layout.addWidget(self.label_button)


        self.bb = QPushButton("Back")
        self.bb.clicked.connect(self.bacKK)  # Placeholder functionality
        layout.addWidget(self.bb)

    def open_upload_page(self):
        self.hide()
        self.upload_page = UploadPage()
        self.upload_page.show()

    #def label_images(self):
    #   QMessageBox.information(self, "Label Images", "Label images functionality to be implemented.")
    def label_images(self):
        if not os.path.exists(uploaded_images_folder):
            QMessageBox.warning(self, "No Images Found", "Upload images first.")
            return
        try:
            self.all_images_window = AllImagesWindow()
            self.all_images_window.show()
        except Exception as e:
            QMessageBox.critical(self, "Error", f"An error occurred: {e}")

    def bacKK(self):
        self.hide()
        self.start = StartingPage()
        self.start.show()


In [13]:
class StartingPage(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setGeometry(200, 200, 300, 300)
        self.setWindowTitle("Starting Page")
        self.initUI()

    def initUI(self):
        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)

        main_layout = QVBoxLayout(central_widget)

        self.label = QLabel("Do you want to see the labelled photos or Unlabelled?")
        self.label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.label.setFont(QFont('Arial', 20))
        main_layout.addWidget(self.label)

        button_layout = QHBoxLayout()
        button_layout.setAlignment(Qt.AlignmentFlag.AlignCenter)

        self.b1 = QPushButton("Labelled")
        self.b1.setFont(QFont('Arial', 16))
        self.b1.clicked.connect(self.show_labelled_photos)  # Connect to show labelled photos method
        button_layout.addWidget(self.b1)

        self.b2 = QPushButton("Unlabelled")
        self.b2.setFont(QFont('Arial', 16))
        self.b2.clicked.connect(self.open_unlabelled_options)
        button_layout.addWidget(self.b2)

        main_layout.addLayout(button_layout)

    def show_labelled_photos(self):
        if not os.path.exists(labelled_images_folder):
            QMessageBox.warning(self, "No Labelled Images", "Label the images first.")
            return
        self.hide()
        try:
            self.labelled_images_window = LabelledImagesWindow(self)
            self.labelled_images_window.show()
        except Exception as e:
            QMessageBox.critical(self, "Error", f"An error occurred: {e}")

    def open_unlabelled_options(self):
        self.hide()
        self.unlabelled_options = UnlabelledOptionsPage()
        self.unlabelled_options.show()


## main

In [ ]:
app = QApplication(sys.argv)
app.setStyleSheet(StyleSheet.style())
main_window = StartingPage()
main_window.show()
sys.exit(app.exec())
